In [2]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.matlib import repmat

In [ ]:
def HyDE_DF(deParameters, otherParameters, initialSolution):
    
    def genpop(a, b, lowMatrix, upMatrix):
        return np.random.uniform(low=lowMatrix, high=upMatrix, size=(a, b))
    
    #-----This is just for notational convenience and to keep the code uncluttered.--------
    I_NP = deParameters.I_NP
    F_weight = deParameters.F_weight
    F_CR = deParameters.F_CR
    I_D = otherParameters.dim     #Number of variables or dimension
    deParameters.nVariables = I_D
    FVr_minbound = otherParameters.lowerlimit * np.array(np.ones((1,otherParameters.dim)))
    FVr_maxbound = otherParameters.upperlimit * np.array(np.ones((1,otherParameters.dim)))
    I_itermax = deParameters.I_itermax
    
    
    #Repair boundary method employed
    BRM = deParameters.I_bnd_constr   #1: bring the value to bound violated
                                      #2: repair in the allowed range
                                      #3: Bounce-back
            
    # Get Objective Function
    fnc= otherParameters.objfun
    
    #-----Check input variables---------------------------------------------
    if I_NP < 5:
        I_NP = 5
        print('I_NP increased to minimal value 5\n')
    
    if (F_CR < 0) | (F_CR > 1):
        F_CR = 0.5
        print('F_CR should be from interval [0, 1] - set to default value 0.5\n')
        
    if I_itermax <= 0:
        I_itermax = 200
        print('I_itermax should be > 0 - set to default value 200\n')
    
    
    #-----Initialize population and some arrays-------------------------------
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # pre-allocation of loop variables
    fitMaxVector = np.empty((1, I_itermax))
    fitMaxVector[:] = np.nan
    
    
    # limit iterations by threshold
    gen = 1; #iterations
    
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #----FM_pop is a matrix of size I_NPx(I_D+1). It will be initialized------
    #----with random values between the min and max values of the-------------
    #----parameters-----------------------------------------------------------
    # FLC modification - vectorization
    minPositionsMatrix = repmat(FVr_minbound, I_NP, 1)
    maxPositionsMatrix = repmat(FVr_maxbound, I_NP, 1)
    deParameters.minPositionsMatrix = minPositionsMatrix
    deParameters.maxPositionsMatrix = maxPositionsMatrix
    
    # generate initial population.
    FM_pop = genpop(I_NP,I_D,minPositionsMatrix,maxPositionsMatrix)
    
    #If you want to inject initial solutions
    if nargin > 2: 
        noInitialSolutions = initialSolution.shape()[0]
        FM_pop(1:noInitialSolutions,:) = initialSolution
        
        
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #------Evaluate the best member after initialization----------------------
    # Modified by FLC
    S_val = feval(fnc, FM_pop)
    I_best_index = np.argmin(S_val); # This mean that the best individual correspond to the best worst performance
    FVr_bestmemit = FM_pop[I_best_index, :] # best member of current iteration
    
    fitMaxVector[:, 0] = S_val(I_best_index) #We save the mean value and mean penalty value
    # The user can decide to save the mean, best, or any other value here
    
    
    
    #------DE-Minimization------------------------------------------------
    #------FM_popold is the population which has to compete. It is--------
    #------static through one iteration. FM_pop is the newly--------------
    #------emerging population.-------------------------------------------
    FVr_rot  = np.arange(0, I_NP, 1)             # rotating index array (size I_NP)
    
    
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #% HYDE self-adaptive parameters
    F_weight_old = repmat(F_weight, I_NP, 3)
    F_weight = F_weight_old
    F_CR_old = repmat(F_CR, I_NP, 1)
    F_CR = F_CR_old
    
    return Fit_and_p,FVr_bestmemit, fitMaxVector